# Class Diagrams

This is a simple viewer for class diagrams.  Customized towards the book.

**Prerequisites**

* _Refer to earlier chapters as notebooks here, as here:_ [Earlier Chapter](Fuzzer.ipynb).

In [ ]:
import bookutils

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from fuzzingbook.ClassDiagram import <identifier>
```

and then make use of the following features.


The `display_class_hierarchy()` function shows the class hierarchy for the given class.  Methods with docstrings (intended to be used by the public) are shown in bold.

```python
>>> display_class_hierarchy(GrammarFuzzer)
```

![](PICS/ClassDiagram-synopsis-1.svg)



## Getting a Class Hierarchy

Using `mro()`, we can access the class hierarchy.  We make sure to avoid duplicates created by `class X(X)`.

In [ ]:
def class_hierarchy(cls):
    superclasses = cls.mro()
    hierarchy = []
    last_superclass_name = ""
    for superclass in superclasses:
        if superclass.__name__ != last_superclass_name:
            hierarchy.append(superclass)
            last_superclass_name = superclass.__name__
    return hierarchy

Here's an example:

In [ ]:
class A_Class:
    """A Class which does A thing right.
    Comes with a longer docstring."""
    def foo(self):
        """The Adventures of the glorious Foo"""
        pass

In [ ]:
class B_Class(A_Class):
    """A subclass inheriting some methods."""
    def foo(self, fight=False):
        """A WW2 foo fighter."""
        pass
    def bar(self, qux=None, bartender=42):
        """A qux walks into a bar.
        `bartender` is an optional attribute."""
        pass

In [ ]:
class C_Class:
    def qux(self):
        pass

In [ ]:
class D_Class(B_Class, C_Class):
    """A subclass inheriting from multiple superclasses.
    Comes with a fairly long, but meaningless documentation."""
    def foo(self):
        B_Class.foo(self)

In [ ]:
class_hierarchy(A_Class)

## Getting a Class Tree

We can use `__bases__` to obtain the immediate base classes.

In [ ]:
D_Class.__bases__

In [ ]:
def class_tree(cls):
    ret = []
    for base in cls.__bases__:
        if base.__name__ == cls.__name__:
            ret += class_tree(base)
        else:
            ret.append((cls, class_tree(base)))
    return ret

In [ ]:
def class_tree(cls):
    def base_tree(base):
        while base.__name__ == cls.__name__:
            base = base.__bases__[0]
        return class_tree(base)

    ret = []
    for base in cls.__bases__:
        ret.append((cls, base_tree(base)))
    return ret

In [ ]:
class_tree(D_Class)

Not perfect yet (multiple instances of same class), but we're getting there.

## Getting methods

In [ ]:
import inspect

In [ ]:
def class_methods(cls):
    def _class_methods(cls):
        all_methods = inspect.getmembers(cls, lambda m: inspect.isfunction(m))
        for base in cls.__bases__:
            all_methods += _class_methods(base)

        return all_methods

    unique_methods = []
    methods_seen = set()
    for (name, fun) in _class_methods(cls):
        if name not in methods_seen:
            unique_methods.append((name, fun))
            methods_seen.add(name)

    return unique_methods

In [ ]:
class_methods(D_Class)

We're only interested in 

* functions _defined_ in that class
* functions that come with a docstring

In [ ]:
def public_class_methods(cls):
    return [(name, method) for (name, method) in class_methods(cls) 
            if method.__qualname__.startswith(cls.__name__)]

In [ ]:
def doc_class_methods(cls):
    return [(name, method) for (name, method) in public_class_methods(cls) 
            if method.__doc__ is not None]

In [ ]:
public_class_methods(D_Class)

In [ ]:
doc_class_methods(D_Class)

## Drawing Class Hierarchy with Method Names

In [ ]:
import html

In [ ]:
import re

In [ ]:
def escape(text):
    text = html.escape(text)
    assert '<' not in text
    assert '>' not in text
    text = text.replace('{', '&#x7b;')
    text = text.replace('|', '&#x7c;')
    text = text.replace('}', '&#x7d;')
    return text

In [ ]:
escape("f(foo={})")

In [ ]:
def escape_doc(docstring):
    DOC_INDENT = 4
    docstring = "&#x0a;".join(
        ' ' * DOC_INDENT + escape(line).strip()
        for line in docstring.split('\n')
    )
    return docstring

In [ ]:
print(escape_doc("'Hello\n    {You|Me}'"))

In [ ]:
from inspect import signature

In [ ]:
def display_class_hierarchy(classes, include_methods=True,
                            project='fuzzingbook'):
    from graphviz import Digraph

    if project == 'debuggingbook':
        CLASS_FONT = 'Raleway, Helvetica, Arial, sans-serif'
        CLASS_COLOR = 'purple'
    else:
        CLASS_FONT = 'Patua One, Helvetica, sans-serif'
        CLASS_COLOR = '#B03A2E'

    METHOD_FONT = "'Fira Mono', 'Source Code Pro', 'Courier', monospace"
    METHOD_COLOR = 'black'

    if isinstance(classes, list):
        starting_class = classes[0]
    else:
        starting_class = classes
        classes = [starting_class]

    title = starting_class.__name__ + " class hierarchy"

    dot = Digraph(comment=title)
    dot.attr('node', shape='record', fontname=CLASS_FONT)
    dot.attr('graph', rankdir='BT', tooltip=title)
    dot.attr('edge', arrowhead='empty')
    edges = set()

    def method_string(method_name, f):
        method_string = f'<font face="{METHOD_FONT}" point-size="10">'
        if f.__doc__ is not None:
            method_string += '<b>' + method_name + '()</b>'
        else:
            method_string += f'<font color="{METHOD_COLOR}">{method_name}()</font>'
        method_string += '</font>'
        return method_string

    def class_methods_string(cls, url):
        methods = public_class_methods(cls)
        # return "<br/>".join([name + "()" for (name, f) in methods])
        if len(methods) == 0:
            return ""

        methods_string = f'<table border="0" cellpadding="0" cellspacing="0" align="left" tooltip="{cls.__name__}" href="#">'
        for show_doc in [True, False]:
            for (name, f) in methods:
                if ((show_doc and f.__doc__ is not None) or
                        (not show_doc and f.__doc__ is None)):

                    method_doc = escape(name + str(inspect.signature(f)))
                    if f.__doc__:
                        method_doc += ":&#x0a;" + escape_doc(f.__doc__)

                    # Tooltips are only shown if a href is present, too
                    tooltip = f' tooltip="{method_doc}"'
                    href = f' href="{url}"'
                    methods_string += f'<tr><td align="left" border="0"{tooltip}{href}>'
                    methods_string += method_string(name, f)
                    methods_string += '</td></tr>'

        methods_string += '</table>'
        return methods_string

    def display_class_node(cls):
        name = cls.__name__
        if cls.__module__ == '__main__':
            url = '#'
        else:
            url = cls.__module__ + '.ipynb'

        if include_methods:
            methods = class_methods_string(cls, url)
            spec = '<{<b><font color="' + CLASS_COLOR + '">' + \
                cls.__name__ + '</font></b>|' + methods + '}>'
        else:
            spec = '<' + cls.__name__ + '>'

        class_doc = escape('class ' + cls.__name__)
        if cls.__doc__:
            class_doc += ':&#x0a;' + escape_doc(cls.__doc__)

        dot.node(name, spec, tooltip=class_doc, href=url)

    def display_class_tree(trees):
        for tree in trees:
            (cls, subtrees) = tree
            display_class_node(cls)
            for subtree in subtrees:
                (subcls, _) = subtree
                if (cls, subcls) not in edges:
                    dot.edge(cls.__name__, subcls.__name__)
                    edges.add((cls, subcls))
            display_class_tree(subtrees)

    for cls in classes:
        tree = class_tree(cls)
        display_class_tree(tree)

    return dot

In [ ]:
display_class_hierarchy([D_Class, A_Class], project='debuggingbook')

In [ ]:
display_class_hierarchy([D_Class, A_Class], project='fuzzingbook')

## Synopsis

The `display_class_hierarchy()` function shows the class hierarchy for the given class (or list of classes).  Methods with docstrings (intended to be used by the public) are shown in bold.

In [ ]:
display_class_hierarchy(D_Class)

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```markdown
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_